In [2]:
import tensorflow as tf
import tensorflow.keras as keras
print(tf.__version__)

2.0.0-alpha0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
n_samples, n_timestamps = 100, 48

# Toy dataset


# Plot the first time series
plt.figure(figsize=(6, 5))
plt.plot(X[0], 'o-')
plt.xlabel('Time', fontsize=14)
plt.title('Plotting a time series', fontsize=16)
plt.tight_layout()
plt.show()